# Configuracion de grafica a usar

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="1";

# Importacion de librerias

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from IPython.display import display, clear_output

%matplotlib inline
import sys
import seaborn as sns
import plotly
import matplotlib.pyplot as plt
sys.path.append('../')
from Datasets_utils.DatasetsLoader import VideoDataGenerator

# Configuraciones para Tensorflow y Keras

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [5]:
tf.debugging.set_log_device_placement(False)

In [6]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

True

# Carga de Datos

In [7]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [8]:
batch_size = 15
original_size = [89,67]
size = [58,58]
frames = 60
canales = 3

In [9]:
dataset = VideoDataGenerator(directory_path = root_path, 
                             batch_size = batch_size, 
                             original_frame_size = original_size, 
                             frame_size=size, 
                             video_frames = frames, 
                             temporal_crop = ('random', 4), 
                             frame_crop = ('random', 2), 
                             shuffle = True, 
                             conserve_original = True)

# Red Neuronal LTC

### Construccion del modelo

In [14]:
#Entrada de la red neuronal
video_shape = tuple([frames]+size+[canales])
dropout = 0.5
entrenamiento = True
lr = 1.#1e-3
ltc_save_path = '/home/jefelitman/Saved_Models/LTC_58x58x60_logsoftmax_rgb_batch15'

In [11]:
entrada = keras.Input(shape=video_shape,
                     batch_size=batch_size,
                     name="Input_video")
#Conv1
x = keras.layers.Conv3D(filters=64, kernel_size=3, padding="same", activation="relu", use_bias=False)(entrada)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2))(x)

#Conv2
x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv3
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv4
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv5
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#fc6
x = tf.reshape(tensor=x, shape=[batch_size,-1], name="Aplanado")
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc7
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc8
x = keras.layers.Dense(len(dataset.to_class), activation="softmax")(x)
salidas = tf.math.log(x)

ltc = keras.Model(entrada, salidas, name="LTC")

In [12]:
#Optimizador del modelo
optimizador = keras.optimizers.SGD(learning_rate=lr)
perdida = keras.losses.SparseCategoricalCrossentropy()

In [ ]:
#keras.utils.plot_model(ltc, 'LTC.png', show_shapes=True)

In [ ]:
ltc = keras.models.load_model(os.path.join(ltc_save_path,'ltc.h5'))

In [13]:
ltc.summary()

Model: "LTC"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_video (InputLayer)     [(15, 60, 58, 58, 3)]     0         
_________________________________________________________________
conv3d (Conv3D)              (15, 60, 58, 58, 64)      5184      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (15, 60, 29, 29, 64)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (15, 60, 29, 29, 128)     221184    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (15, 30, 14, 14, 128)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (15, 30, 14, 14, 256)     884736    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (15, 15, 7, 7, 256)       0       

### Entrenamiento de la red

In [15]:
def precision(y, y_predicho):
    clases = tf.argmax(y_predicho, axis=1)
    comparaciones = tf.equal(y, clases)
    return tf.reduce_mean(tf.cast(comparaciones, tf.int32))

In [16]:
step = 1
train_epoch = 1
train_loss = []
train_accuracy = []
maximo = -1
step_max = 0
while train_epoch <= 1:
    with tf.device('/CPU:0'):
        if step % 1000 == 0:
            ltc.save(os.path.join(ltc_save_path,'ltc.h5'))
            print('Modelo salvado en la iteracion: {i}'.format(i=step))

        if step == 80000//batch_size or step == 125000//batch_size:
            lr = lr * 0.1
            optimizador = keras.optimizers.SGD(learning_rate=lr)
    
    batch, labels = dataset.get_next_train_batch(canales)
    
    with tf.GradientTape() as tape:
        predicciones = ltc(batch)
        valor_perdida = perdida(labels, predicciones)
    
    with tf.device('/CPU:0'):
        train_loss.append(valor_perdida.numpy())    
        train_accuracy.append(precision(labels, predicciones).numpy())

        if train_accuracy[-1] >= maximo:
            maximo = train_accuracy[-1]
            step_max = step

        clear_output(wait=True)
        print("Step: ",step)
        print("Train Epoch: ",train_epoch, " Train batch: ",dataset.train_batch_index,"/",dataset.train_batches)
        print("Train_Loss: ",train_loss[-1]," Train_Acuraccy: ",train_accuracy[-1])
        print("Tain_max_Acuraccy: {m} at the step: {s}".format(m=maximo, s=step_max))
        
        if dataset.train_batch_index == dataset.train_batches:
            train_epoch += 1
        
    grads = tape.gradient(valor_perdida, ltc.trainable_weights)
    
    optimizador.apply_gradients(zip(grads, ltc.trainable_weights))
    
    step += 1

Step:  12745
Train Epoch:  1  Train batch:  12745 / 12745
Train_Loss:  4.6151204  Train_Acuraccy:  0
Tain_max_Acuraccy: 0 at the step: 12745


In [23]:
#Comprobacion de que mi modelo esta aprendiendo o variando las capas
from copy import deepcopy
pesos_antes = deepcopy(ltc.get_layer(index=3).get_weights()[0])
pesos_antes.shape

(3, 3, 3, 64, 128)

In [28]:
import numpy as np
np.all(pesos_antes == ltc.get_layer(index=3).get_weights()[0])

False

In [33]:
ltc.get_layer(index=3).get_weights()[0][:,:,:,1,0]

array([[[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]]], dtype=float32)

### Graficas de los resultados de entrenamiento

In [ ]:
fig = plt.figure()
plt.plot(range(len(train_loss)), train_loss,'k--')
plt.title('Loss over steps')
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_loss.png'))

In [ ]:
fig = plt.figure()
plt.plot(range(len(train_accuracy)), train_accuracy,'k--')
plt.title('Accuracy over steps')
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_accuracy.png'))

### Evaluacion del entrenamiento

In [ ]:
#Desactivo las capas de dropout
ltc.get_layer(index=13)._trainable = False
ltc.get_layer(index=15)._trainable = False

In [ ]:
step = 1
test_epoch = 1
test_loss = []
test_accuracy = []
entrenamiento = False
while test_epoch == 1:
    
    batch, labels = dataset.get_next_test_batch(canales)
    
    with tf.GradientTape() as tape:
        predicciones = ltc(batch)
        valor_perdida = perdida(labels, predicciones)
    
    with tf.device('/CPU:0'):
        test_loss.append(valor_perdida.numpy())    
        test_accuracy.append(precision(labels, predicciones).numpy())
    
        clear_output(wait=True)
        print("Step: ",step)
        print("Test Epoch: ",test_epoch, " Test batch: ",dataset.test_batch_index,"/",dataset.test_batches)
        print("Test_Loss: ",test_loss[-1]," Test_Acuraccy: ",test_accuracy[-1])
        
        if dataset.test_batch_index == dataset.test_batches:
        test_epoch += 1
    
    step += 1

### Graficas de los resultados de entrenamiento

In [ ]:
fig = plt.figure()
plt.plot(range(len(test_loss)), test_loss,'k--')
plt.title('Loss over steps')
plt.show()
fig.savefig(os.path.join(ltc_save_path,'test_loss.png'))

In [ ]:
fig = plt.figure()
plt.plot(range(len(test_accuracy)), test_accuracy,'k--')
plt.title('Accuracy over steps')
plt.show()
fig.savefig(os.path.join(ltc_save_path,'test_accuracy.png'))